<a href="https://colab.research.google.com/github/flavio-campos/Veritas-desafio-Alura/blob/main/Veritas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Veritas* - Detector de textos enganosos

Imagem

## Veritás: Desvendando a Verdade por Trás das Palavras
O sistema Veritás - Detector de Textos Enganosos tem como propósito analisar criticamente textos para identificar e expor técnicas de manipulação e desinformação. Ele funciona como um "detector de mentiras" textual, examinando as entranhas de um documento para revelar possíveis falácias lógicas, exageros, apelos emocionais, informações falsas, linguagem discriminatória, vieses, falta de evidências e outros elementos que comprometem a veracidade e a confiabilidade da informação.

**Objetivo:** Oferecer aos usuários uma ferramenta poderosa para navegar o cenário informacional complexo da atualidade, permitindo-lhes discernir entre conteúdo confiável e manipulador.

**Funcionamento:** O sistema utiliza um prompt de 10 passos para dissecar o texto, funcionando como um guia para a análise crítica. Cada passo foca em um aspecto específico da comunicação, buscando identificar potenciais problemas e vieses que podem influenciar a interpretação do leitor.

**Resultados:** O Veritás fornece um relatório detalhado com os resultados da análise, apontando os elementos problemáticos encontrados no texto e oferecendo uma avaliação crítica da sua confiabilidade.

Em resumo, o Veritás empodera os leitores a questionarem o que leem, promovendo um consumo de informação mais consciente e crítico.

<iframe src="https://drive.google.com/file/d/1oOni33falCQBheyx9QgTUspHPBL_ujhe/preview" width="640" height="480" allow="autoplay"></iframe>

## Instruções de uso

Inicialmente configure sua API do Google AI Studio na seção Secrets do Colab e o arquivo a ser criticado na área de arquivos. Então execute todos os passos sequencialmente, até o último que apresentará o resultado da análise na seção de resultados do Colab. Veja mais sobre isto abaixo em *Pré-requisitos*.

## Restrições da versão atual:
* O documento deve estar no formato PDF

## Ideias de melhorias futuras:
* Reimplementar a ferramenta como um pluggin do Chrome e outros browsers, para que possa fazer críticas das páginas HTML carregadas, assim proporcionando uma visão desnuda dos textos lidos;
* Possiblidades de integrar também ferramentas para smartfones e leitores de e-mail, para analisar e-mails e mensagens recebidas de mensageiros. No caso de mensagens, dando possibilidade para analisar um conjunto de mensagens dentro de uma conversa, em um só contexto.

### *Pré-requisitos*

Para funcionar corretamente, este notebook exige dois pré-requisitos:

1.   Sua chave de API do Google AI Studio, deve estar armazenada no Colab como um Secrets, de nome MyGoogleAPIKey. Para acessar a área de secrets, acesse o ícono de chave na barra lateral esquerda do Colab;
2.   Você deve disponibilizar, através da seção de Arquivos do Colab, um PDF no formato texto. Você pode acessar esta seção, pelo ícone de pasta, também na barra lateral esquerda. Caso o nome do arquivo seja grande ou complexo, considere renomeá-lo para algo mais simples. Obs.: O próprio colab permite renomear o arquivo disponibilizado.



# Instalando os pacotes necessários ao funcionamento do sistema

In [1]:
!pip install -q -U google-generativeai
!pip install -q -U pypdf

Importando os pacotes que serão utilizados na implementação

In [2]:
from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import userdata
from pypdf import PdfReader
import json
import textwrap
from IPython.display import display
from IPython.display import Markdown
import os

Parâmetros de configuração da API

In [3]:
generation_config = {
  "temperature": 1
}

safety_settings = [
  { "category": "HARM_CATEGORY_HARASSMENT",        "threshold": "BLOCK_MEDIUM_AND_ABOVE" },
  { "category": "HARM_CATEGORY_HATE_SPEECH",       "threshold": "BLOCK_MEDIUM_AND_ABOVE" },
  { "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE" },
  { "category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE" },
]


Configurando a API e o modelo utilizado

In [4]:
MY_GOOGLE_API_KEY = userdata.get('MyGoogleAPIKey')

genai.configure(api_key=MY_GOOGLE_API_KEY)

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


Função auxiliar para quebre de PDF em partes

In [5]:
def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]

  # Quabra o PDF em páginas para enviá-lo por partes para o modelo
  reader = PdfReader(pathname)
  pages = [page.extract_text() for page in reader.pages]
  for index, page in enumerate(pages):
    parts.append(f"--- PAGE {index} ---")
    parts.append(page)
  return parts

Obtendo caminho e nome do documento PDF a ser analisado. Se for feito um upload direto na seção de arquivos do Colab, sem criar ou entrar em subpastas, considere colocar só o nome do arquivo.

In [6]:
documento = input("Entre com o caminho e nome do documento a ser analisado: ")

while not os.path. exists(documento):
  print(f"Não foi possível encontrar o arquivo {documento}. Certifique-se do caminho e entre novamente, por favor!\n")
  documento = input("Entre com o caminho e nome do documento a ser analisado: ")


Entre com o caminho e nome do documento a ser analisado: documento.pdf


In [9]:
convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": extract_pdf_pages(documento)
  },
])

convo.send_message('Faça uma avaliação crítica do documento apresentado.\n1. Identifique e explique qualquer uso de falácias lógicas neste texto;\n2. Analise o texto para exageros ou generalizações amplas e forneça exemplos;\n3. Detecte e discuta quaisquer apelos emocionais presentes no texto;\n4. Verifique a presença de informações falsas ou enganosas e indique correções necessárias;\n5. Examine o texto em busca de linguagem retórica discriminatória e explique o impacto potencial disso;\n6. Avalie o texto para qualquer viesses implícitos ou explícitos e discuta sua influência na mensagem geral;\n7. Procure por afirmações sem evidências ou fontes confiáveis e solicite a verificação dessas afirmações;\n8. Analise o uso de metáfora e analogias para determinar se elas são apropriadas e precisas;\n9. Faça uma avaliação crítica da estrutura argumentativa do texto;\n10. Revele e discuta quaisquer contradições internas ou inconsistência no texto.\n\nSeu retorno deve ser em formato json para ser atribuído a uma variável python, seguindo o exemplo abaixo: {\n "Análise Crítica do Texto": "O texto apresenta um chamado à ação urgente para combater as mudanças climáticas, utilizando as recentes tragédias no Rio Grande do Sul como catalisador. Apesar de levantar pontos importantes, a análise crítica revela diversas falhas argumentativas e estratégias retóricas questionáveis.",\n "Falácias Lógicas": [\n {\n "nome": "Falsa dicotomia",\n "explicação": "O texto apresenta a questão climática como uma aposta binária: ou as previsões apocalípticas estão certas, ou erradas. Essa simplificação ignora a complexidade da questão e a gama de cenários possíveis, desconsiderando consequências intermediárias.",\n "exemplo": "Aposta de Pascal"\n },\n {\n "nome": "Apelo à emoção",\n "explicação": "A descrição das tragédias no Rio Grande do Sul, com ênfase no número de mortos e na imprevisibilidade dos eventos, busca gerar medo e urgência no leitor, predispondo-o a concordar com a necessidade de ação imediata, sem aprofundar a análise crítica das soluções propostas.",\n "exemplo": "Descrição das tragédias no RS"\n },\n {\n "nome": "Generalização Apressada",\n "explicação": "O texto usa as chuvas no RS como evidência inquestionável da necessidade de um \"Projeto Manhattan\" para o clima. Essa generalização ignora outros fatores que contribuem para desastres naturais, como planejamento urbano inadequado e falta de investimento em infraestrutura, simplificando a problemática a um único fator: mudança climática.",\n "exemplo": "Exemplo do RS"\n }\n ],\n "Exageros e Generalizações": [\n {\n "exemplo": "\"Plano Marshall\" para o RS",\n "explicação": "A comparação com o Plano Marshall, que reconstruiu a Europa após a Segunda Guerra Mundial, é exagerada. A situação do RS, apesar de grave, não se compara à escala de destruição da guerra, e o uso dessa analogia inflaciona a urgência da situação."\n },\n {\n "exemplo": "\"Projeto Manhattan\" para o clima",\n "explicação": "A analogia ao Projeto Manhattan, que desenvolveu a bomba atômica, é igualmente problemática.  O texto sugere que precisamos de um esforço de mesma magnitude, com a mesma urgência e foco, para lidar com a mudança climática. Essa comparação, além de exagerada, romantiza um projeto com consequências catastróficas, ignorando os dilemas éticos envolvidos."\n }\n ],\n "Apelos Emocionais": [\n {\n "exemplo": "Tragédia no RS",\n "explicação": "A descrição das mortes e dos impactos no estado visa sensibilizar o leitor, evocando sentimentos de compaixão e medo."\n },\n {\n "exemplo": "\"Corrida existencial\"",\n "explicação": "A analogia com a corrida armamentista durante a Guerra Fria busca incutir um senso de urgência e  ameaça iminente,  sugerindo que estamos em uma \"guerra\" contra o clima."\n }\n ],\n "Informações Falsas ou Enganosas": [\n {\n "afirmação": "\"Emissões humanas representam menos de 4% do total da natureza\"",\n "correção": "Essa afirmação, presente em um comentário, é imprecisa e carece de fontes confiáveis. A contribuição humana para o efeito estufa é significativa e comprovadamente impacta o clima global."\n }\n ],\n "Linguagem Retórica Discriminatória": {\n "presença": "Não há elementos de linguagem discriminatória no texto principal. Entretanto, os comentários podem apresentar esse tipo de linguagem e devem ser moderados pela plataforma para garantir um ambiente respeitoso e inclusivo."\n },\n "Vieses": [\n {\n "tipo": "Viés de confirmação",\n "explicação": "O texto seleciona exemplos e argumentos que reforçam a tese da urgência no combate à mudança climática, sem apresentar visões alternativas ou dados que possibilitem uma análise mais balanceada."\n },\n {\n "tipo": "Viés de autoridade",\n "explicação": "O texto cita figuras políticas como Lula e Rodrigo Pacheco para endossar a necessidade de ação, sem questionar criticamente as propostas ou as reais intenções por trás delas."\n }\n ],\n "Afirmações sem Evidências": [\n {\n "afirmação": "\"Energia limpa e preservação ambiental não são incompatíveis com desenvolvimento econômico\"",\n "verificação": "Essa afirmação, embora plausível, carece de embasamento e exemplos concretos. O texto assume essa premissa como verdade absoluta, sem apresentar dados ou estudos que a comprovem."\n },\n {\n "afirmação": "Eficácia do Plano Nacional de Proteção e Defesa Civil",\n "verificação": "O texto menciona o plano em desenvolvimento, mas não apresenta detalhes concretos sobre seus objetivos, estratégias e orçamento, tornando impossível avaliar sua real capacidade de prevenir desastres."\n }\n ],\n "Metáforas e Analogias": [\n {\n "analogia": "Aposta de Pascal",\n "avaliação": "A analogia é inadequada, pois simplifica a questão climática a uma escolha binária, ignorando a complexidade dos fatores envolvidos e a gama de cenários possíveis."\n },\n {\n "analogia": "Plano Marshall e Projeto Manhattan",\n "avaliação": "As analogias são exageradas e inadequadas, pois comparam a situação do RS e a questão climática a eventos históricos de escala e magnitude distintas, inflacionando a urgência e simplificando as soluções."\n }\n ],\n "Estrutura Argumentativa": "O texto apresenta uma estrutura argumentativa frágil, baseada em falácias, apelos emocionais e analogias inadequadas. A falta de dados concretos, fontes confiáveis e análises aprofundadas enfraquece a argumentação e compromete a confiabilidade das conclusões.",\n "Contradições e Inconsistências": [\n {\n "contradição": "O texto critica a falta de investimento em prevenção, mas não apresenta uma análise crítica das propostas de \"Plano Marshall\" e \"Projeto Manhattan\", que podem focar apenas na reconstrução, repetindo erros do passado."\n },\n {\n "contradição": "A ênfase na urgência e na necessidade de ação imediata contrasta com a falta de detalhes concretos sobre as soluções propostas, seus custos e impactos a longo prazo."\n }\n ],\n "Conclusão": "O texto, apesar de levantar a importante questão da mudança climática, apresenta diversas falhas argumentativas e estratégias retóricas questionáveis. A utilização de falácias, exageros, apelos emocionais e analogias inadequadas compromete a validade das conclusões e limita o debate sobre soluções eficazes para a problemática.",\n "Recomendações": [\n "Substituir as analogias por dados concretos e estudos científicos que demonstrem a urgência da questão climática.",\n "Apresentar soluções concretas, com análise de custos e benefícios a curto e longo prazo, considerando diferentes cenários e impactos socioeconômicos.",\n "Abrir espaço para visões alternativas e fomentar um debate plural, baseado em evidências e  livre de vieses."\n ]\n }\n')
json_data = convo.last.text.replace('```json', '').replace('```','')

In [10]:
# Converte o JSON para um dicionário Python
#data = json.loads(json.dumps(json_data))
data = json.loads(json_data)

# TODO: Converter a série dechamas display(to_markdown()) em uma função recursiva que imprime a esturuta percorerndo a árvore

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Imprime a análise formatada em Markdown
display(to_markdown("# Análise Crítica do Texto"))
display(to_markdown(data["Análise Crítica do Texto"]))
display(to_markdown("\n## Falácias Lógicas"))
for falacia in data["Falácias Lógicas"]:
    display(to_markdown(f"  * **{falacia['nome']}:** {falacia['explicação']} *Exemplo: {falacia['exemplo']}*"))
display(to_markdown("\n## Exageros e Generalizações"))
for exagero in data["Exageros e Generalizações"]:
    display(to_markdown(f"  * **{exagero['exemplo']}:** {exagero['explicação']}"))
display(to_markdown("\n## Apelos Emocionais"))
for apelo in data["Apelos Emocionais"]:
    display(to_markdown(f"  * **{apelo['exemplo']}:** {apelo['explicação']}"))
display(to_markdown("\n## Informações Falsas ou Enganosas"))
for info in data["Informações Falsas ou Enganosas"]:
    display(to_markdown(f"  * **Afirmação:** {info['afirmação']}"))
    display(to_markdown(f"  * **Correção:** {info['correção']}"))
display(to_markdown("\n## Linguagem Retórica Discriminatória"))
display(to_markdown(data["Linguagem Retórica Discriminatória"]["presença"]))
display(to_markdown("\n## Vieses"))
for vies in data["Vieses"]:
    display(to_markdown(f"  * **{vies['tipo']}:** {vies['explicação']}"))
display(to_markdown("\n## Afirmações sem Evidências"))
for afirmacao in data["Afirmações sem Evidências"]:
    display(to_markdown(f"  * **Afirmação:** {afirmacao['afirmação']}"))
    display(to_markdown(f"  * **Verificação:** {afirmacao['verificação']}"))
display(to_markdown("\n## Metáforas e Analogias"))
for metafora in data["Metáforas e Analogias"]:
    display(to_markdown(f"  * **Analogia:** {metafora['analogia']}"))
    display(to_markdown(f"  * **Avaliação:** {metafora['avaliação']}"))
display(to_markdown("\n## Estrutura Argumentativa"))
display(to_markdown(data["Estrutura Argumentativa"]))
display(to_markdown("\n## Contradições e Inconsistências"))
for contradicao in data["Contradições e Inconsistências"]:
    display(to_markdown(f"  * {contradicao['contradição']}"))
display(to_markdown("\n## Conclusão"))
display(to_markdown(data["Conclusão"]))
display(to_markdown("\n## Recomendações"))
for recomendacao in data["Recomendações"]:
    display(to_markdown(f"  * {recomendacao}"))


> # Análise Crítica do Texto

> O texto apresenta um chamado à ação urgente para combater as mudanças climáticas, utilizando as recentes tragédias no Rio Grande do Sul como catalisador. Apesar de levantar pontos importantes, a análise crítica revela diversas falhas argumentativas e estratégias retóricas questionáveis, como o uso de falácias, analogias inadequadas, generalizações e apelos emocionais.

> 
> ## Falácias Lógicas

>   * **Falsa dicotomia:** O texto apresenta a questão climática como uma aposta binária: ou as previsões apocalípticas estão certas, ou erradas, desconsiderando a complexidade da questão e a gama de cenários possíveis. *Exemplo: Aposta de Pascal*

>   * **Apelo à emoção:** A descrição das tragédias no Rio Grande do Sul, com ênfase no número de mortos, busca gerar medo e urgência no leitor, predispondo-o a concordar com a necessidade de ação imediata, sem aprofundar a análise crítica das soluções propostas. *Exemplo: Descrição das tragédias no RS*

...

> O texto, apesar de levantar a importante questão da mudança climática, apresenta diversas falhas argumentativas e estratégias retóricas questionáveis. A utilização de falácias, exageros, apelos emocionais e analogias inadequadas compromete a validade das conclusões e limita o debate sobre soluções eficazes para a problemática.

> 
> ## Recomendações

>   * Substituir as analogias por dados concretos e estudos científicos que demonstrem a urgência da questão climática.

>   * Apresentar soluções concretas, com análise de custos e benefícios a curto e longo prazo, considerando diferentes cenários e impactos socioeconômicos.

>   * Abrir espaço para visões alternativas e fomentar um debate plural, baseado em evidências e  livre de vieses.